Example place fields from each group

In [ ]:
import scipy.stats as s
import numpy as np
import matplotlib.pyplot as plt
from os import path
import seaborn as sns
import pandas as pd

# Import project specific modules and enable automatic reloading
import sys
from pathlib import Path
import os
eraser_path = Path(os.getcwd()).parent
reinstatement_path = eraser_path.parent / 'FearReinstatement'
sys.path.append(str(eraser_path))
sys.path.append(str(reinstatement_path))
%load_ext autoreload
%autoreload 2

import eraser_reference as err
import er_plot_functions as er
from plotting import Fig, pretty_plot, FigMirror
import placefield_stability as pfs
import Placefields as pf
import discrimination as discr
import ca_traces as trc

Top rows: 8 fields from learner before shock and after shock. Top row = raw behavior with red dots, next row = PFs, next row = after PFS, bottom row = raw after behavior with red dots. Left 4 = Open, Right 4 = Shock

Next rows: Non-learner, same as above

Bottom rows: ANI, same as above